In [1]:
!pip install pymongo

In [2]:
!pip install python-dotenv

In [3]:
!pip install mysql-connector-python

In [1]:
import json
import os
import mysql.connector
import pymongo
import mysql.connector

from pymongo import MongoClient
from bson import ObjectId
from dotenv import load_dotenv
from datetime import datetime
from pprint import pprint
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from datetime import datetime

In [2]:
load_dotenv()

True

## MONGODB

In [3]:
mongo_username = os.environ.get("MONGOUSERNAME")
mongo_password = os.environ.get("MONGOPASSWORD")

uri = "mongodb+srv://{}:{}@twitter.qlewowk.mongodb.net/?retryWrites=true&w=majority&appName=twitter".format(mongo_username, mongo_password)

client = MongoClient(uri, server_api=ServerApi('1'))

try:
    client.admin.command('ping')
    print("Connected to MongoDB!")
except Exception as e:    print(e)

Connected to MongoDB!


In [94]:
db = client["sample_test"]
collection = db["tweets_test"]

In [86]:
file_name = "corona-out-2"

In [6]:
lines=0
num_tweets=0
num_retweets=0
print_tweet = 0
print_retweet = 0
tweets = {}

with open(file_name, "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            lines = lines + 1
#             if data['is_quote_status']:
#                 pprint(data)
#                 pprint(data['text'])
#                 pprint(data['user']['name'])
#                 print("****************** quoted")
# #                 pprint(data['quoted_status'])
#                 pprint(data['quoted_status']['text'])
#                 pprint(data['quoted_status']['user']['name'])
#                 print("****************** retweeted")
# #                 pprint(data['retweeted_status'])
#                 pprint(data['retweeted_status']['text'])
#                 pprint(data['retweeted_status']['user']['name'])
#                 print("******************")
#                 print("******************")
#                 print("******************")
            if ( data['text'].startswith('RT') ):
                num_retweets += 1
#                 if (print_retweet < 2):
#                     print_retweet += 1
#                     print('retweetstatus--',data['retweeted_status'],'\n')
                if data['retweeted_status']['text'].startswith('RT'):
                    print("booooyaaa")
                print('RETWEET\n','id--',data['id'],'text--',data['text'],'\n')
                print('User-- ',data['user']['name'],'\n')
                
                print('OG TWEET\n','id--',data['retweeted_status']['id'],'text--',data['retweeted_status']['text'],'\n')
                print('User-- ',data['retweeted_status']['user']['name'],'\n')
                print("***************")
            else:
                num_tweets += 1
                if (print_tweet < 2):
                    print_tweet += 1
#                     print('TWEET\n','id--',data['id'],'text--',data['text'],'\n')
#                     print('User-- ',data['user'],'\n')
#                     print('retweetstatus--',data['retweeted_status'],'\n')
            if (data['id_str'] not in tweets):
                tweets[data['id_str']] = data
        except:
            continue

print('num of lines=',lines, 'num of tweets=',num_tweets, 'num of retweets=',num_retweets)
print('num of unique tweets/retweets=',len(tweets.keys()))

RETWEET
 id-- 1249403767180668930 text-- RT @nuffsaidny: wishing death on people is weirdo behavior. 

User--  juwelz v 

OG TWEET
 id-- 1249378751349231616 text-- wishing death on people is weirdo behavior. 

User--  NUFF 

***************
RETWEET
 id-- 1249403768023678982 text-- RT @lale_karanfil: In Turkey, there are 300 thousand prisoners and 150 thousand prison employees in prisons. 
Prisons are the most risky pl… 

User--  efe09 

OG TWEET
 id-- 1249397541596286979 text-- In Turkey, there are 300 thousand prisoners and 150 thousand prison employees in prisons. 
Prisons are the most ris… https://t.co/GogsgfUxkE 

User--  Karanfil Lale 

***************
RETWEET
 id-- 1249403769193779202 text-- RT @umesh_agr: Thank You Sir !!
Really thankful for encouraging the 
@BSNL_OR 
 Staff risking their life in maintaining the Telecom Service… 

User--  Ravin Gupta 

OG TWEET
 id-- 1249326224964345857 text-- Thank You Sir !!
Really thankful for encouraging the 
@BSNL_OR 
 Staff risking their l

RETWEET
 id-- 1249404449975566341 text-- RT @gergerliogluof: Bu genç kadın kanser hastası ve salgından dolayı kemoterapi göremiyor!

Corona'dan mı ölsün kanser hastalığından mı..!?… 

User--  VAHŞİ-EBU HARB 

OG TWEET
 id-- 1249348068106997762 text-- Bu genç kadın kanser hastası ve salgından dolayı kemoterapi göremiyor!

Corona'dan mı ölsün kanser hastalığından mı… https://t.co/HB2zE0xJQN 

User--  Ömer Faruk Gergerlioğlu 

***************
RETWEET
 id-- 1249404450688638976 text-- RT @NJeanKitenge: Hey @policefederale je pense que quelques explications s’imposent. Woluwe saint-lambert c’est déjà 125 cas détecté de cor… 

User--  Sarah 

OG TWEET
 id-- 1249384339584028672 text-- Hey @policefederale je pense que quelques explications s’imposent. Woluwe saint-lambert c’est déjà 125 cas détecté… https://t.co/4Nq7Lwwun6 

User--  Ndjakala J. Kitenge 

***************
RETWEET
 id-- 1249404451439251457 text-- RT @__Effendi__: Coba deh debat @jokowi vs @aniesbaswedan soal penanganan wabah coron

RETWEET
 id-- 1249405274127060993 text-- RT @NpT6HnBWMfs7L8w: "وعلى كلّ حال لقد كذّب المكذّبون بحديث الله سواء الملحِدون أو المشركون أو المسلمون إلا مَن رِحم ربّي منهم وكسب في إيما… 

User--  ابراهيم بن اسماعيل 

OG TWEET
 id-- 1249397380384002050 text-- "وعلى كلّ حال لقد كذّب المكذّبون بحديث الله سواء الملحِدون أو المشركون أو المسلمون إلا مَن رِحم ربّي منهم وكسب في إ… https://t.co/0A8kYsGDa0 

User--  ابوعامر العباسي 

***************
RETWEET
 id-- 1249405275993407488 text-- RT @djeppink: Dit is de Derde wereldstad, Brussel, hoofdstad van België, Vlaanderen en EU. Helft Derde wereld; andere helft Eerste wereld.… 

User--  Eric Jorem 

OG TWEET
 id-- 1249310360450076680 text-- Dit is de Derde wereldstad, Brussel, hoofdstad van België, Vlaanderen en EU. Helft Derde wereld; andere helft Eerst… https://t.co/2KS48NSuc5 

User--  Derk Jan Eppink 

***************
RETWEET
 id-- 1249405276488376320 text-- RT @LambaAlka: वाह संधियों - अंधभक्तों 
एक खिलाड़ी जो BJP की टिकट पर चुनाव हारने के बाद 

RETWEET
 id-- 1249405896997789698 text-- RT @VIVAcoid: Kabar Gembira, Semua Pasien Positif Corona di Aceh Sembuh https://t.co/dVhrMXT3pW 

User--  Andi.A.Prabowo 

OG TWEET
 id-- 1249313957094912012 text-- Kabar Gembira, Semua Pasien Positif Corona di Aceh Sembuh https://t.co/dVhrMXT3pW 

User--  VIVAcoid 

***************
RETWEET
 id-- 1249405897064845312 text-- RT @illegaldaaru: BB is over guys

Toh move on kro

Trolling sahi hai par ek hadd mein 

Iss waqt hum life mein sabse badi problem face kar… 

User--  bhuwan joshi 

OG TWEET
 id-- 1249375729831366658 text-- BB is over guys

Toh move on kro

Trolling sahi hai par ek hadd mein 

Iss waqt hum life mein sabse badi problem fa… https://t.co/9jUl2mGgg4 

User--  Daaru✍❤️ 

***************
RETWEET
 id-- 1249405899124412418 text-- RT @TheCourtKim: It’s Easter Sunday... 

User--  🇬🇭putaonika 

OG TWEET
 id-- 1249321591655235584 text-- It’s Easter Sunday... 

User--  𝕮𝖔𝖚𝖗𝖙 𝕶𝖎𝖒 💥 

***************
RETWEET
 id-- 1249405899602444289 text--

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
pprint(data)

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sun Apr 12 18:47:53 +0000 2020',
 'entities': {'hashtags': [],
              'symbols': [],
              'urls': [{'display_url': 'dlvr.it/RTf5SD',
                        'expanded_url': 'http://dlvr.it/RTf5SD',
                        'indices': [75, 98],
                        'url': 'https://t.co/eDbMsxFA84'}],
              'user_mentions': [{'id': 47596019,
                                 'id_str': '47596019',
                                 'indices': [3, 18],
                                 'name': 'Liputan6.com',
                                 'screen_name': 'liputan6dotcom'}]},
 'favorite_count': 0,
 'favorited': False,
 'filter_level': 'low',
 'geo': None,
 'id': 1249408918196871168,
 'id_str': '1249408918196871168',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 

### approach 1

In [8]:

tweet_fields = ["id_str", "created_at", "favorited", "favorite_count", "lang", "retweet_count",
                "quote_count", "reply_count", "retweet_count", "entities", "text"]

def get_tweets(file_name, retweets = False):
    lines=0
    num_tweets=0
    num_retweets=0
    print_tweet = 0
    print_retweet = 0
    tweets = []
    exceptions = []
    unique_ones = set()
    with open(file_name, "r") as f1:
        for line in f1:
            try:
                data = json.loads(line)
                final_data = {}
                if data["id"] not in unique_ones:
                    unique_ones.add(data["id"])
                    lines = lines + 1
                    final_data["_id"] = data["id"]
                    final_data["user_id"] = data["user"]["id"]
                    for field in tweet_fields:
                        final_data[field] = data[field]
                    if (data['text'].startswith('RT')):
                        num_retweets += 1
                        if retweets and num_retweets == 1:
                                pprint(data["retweeted_status"])
                        retweeted_status = {}
                        retweeted_status["_id"] = data["retweeted_status"]["id"]
                        retweeted_status["user_id"] = data["retweeted_status"]["user"]["id"]
                        for field in tweet_fields:
                            retweeted_status[field] = data["retweeted_status"][field]
                        final_data["retweeted_status"] = retweeted_status
                        final_data["retweeted"] = True
                    else:
                        num_tweets += 1
                    if final_data:
                        tweets.append(final_data)
            except Exception as e:
                exceptions.append(e)

    print('num of lines=',lines, 'num of tweets=',num_tweets, 'num of retweets=',num_retweets)
    print(len(tweets),  " tweets captured")
    print(len(exceptions), " exceptions")
    return tweets

### approach 2

In [11]:
metric_fields = ["id_str", "created_at", "favorited", "favorite_count", "lang",
                "quote_count", "reply_count", "retweet_count", "entities", "text"]


def get_tweets(file_name):
    lines = 0
    num_tweets = 0
    num_retweets = 0
    num_quotes = 0
    tweets = []
    exceptions = []
    unique_ones = set()
    with open(file_name, "r") as f1:
        for line in f1:
            try:
                data = json.loads(line)
                final_data = {}
                retweeted_status = {}
                quoted_status = {}
                # collect unique tweets
                if data["id"] not in unique_ones:
                    unique_ones.add(data["id"])
                    lines = lines + 1

                    # Retweets 
                    if (data['text'].startswith('RT')):
                        num_retweets += 1

                        #current tweet user and metrics
                        final_data["is_retweeted"] = True
                        final_data["_id"] = data["id"]
                        final_data["user_id"] = data["user"]["id"]
                        final_data["is_quote_status"] = False
                        for field in metric_fields:
                            final_data[field] = data[field]
                        
                        # original tweet
                        data_level_1 = data["retweeted_status"]
                        final_data["text"] = data_level_1["text"]
                        retweeted_status["_id"] = data_level_1["id"]
                        retweeted_status["user_id"] = data_level_1["user"]["id"]
                        
                        for field in metric_fields:
                            retweeted_status[field] = data_level_1[field]

                        #quotes
                        if data_level_1['is_quote_status']:
                            data_level_2 = data_level_1['quoted_status']
                            data_level_1["is_quote_status"] = True
                            quoted_status["_id"] = data_level_2["id"]
#                             quoted_status["text"] = data_level_2["text"]
                            quoted_status["user_id"] = data_level_2["user"]["id"]
                            
                            for field in metric_fields:
                                quoted_status[field] = data_level_2[field]

                        else:
                            data_level_1["is_quote_status"] = False

                        retweeted_status["quoted_status"] = quoted_status
                        final_data["retweeted_status"] = retweeted_status

                    else:
                        num_tweets += 1
                        data_level_1 = data

                        # normal tweet
                        final_data["_id"] = data_level_1["id"]
#                         final_data["text"] = data_level_1["text"]
                        final_data["user_id"] = data_level_1["user"]["id"]
                        for field in metric_fields:
                            final_data[field] = data_level_1[field]
                        
                        #quotes
                        if data_level_1["is_quote_status"]:
                            data_level_2 = data_level_1["quoted_status"]
                            data_level_1["is_quote_status"] = True
                    
                            quoted_status["_id"] = data_level_2["id"]
#                             quoted_status["text"] = data_level_2["text"]
                            quoted_status["user_id"] = data_level_2["user"]["id"]
                            
                            final_data["quoted_status"] = quoted_status
                            for field in metric_fields:
                                quoted_status[field] = data_level_2[field]
                        
                        else:
                            data_level_1["is_quote_status"] = False

                    if final_data:
                        tweets.append(final_data)

            except Exception as e:
#                 print(e)
                exceptions.append(e)

    print('num of lines=',lines, 'num of tweets=',num_tweets, 'num of retweets=',num_retweets)
    print(len(tweets),  " tweets captured")
    print(len(exceptions), " exceptions")
    return tweets

### approach 3

In [106]:
tweet_fields = ["id_str", "created_at","favorite_count", "lang",
                "quote_count", "reply_count", "retweet_count", "entities"]

def get_og_tweets(file_name):
    num_tweets=0
    num_retweets=0
    tweets = {}
    exceptions = []
    retweeted_status = {}

    with open(file_name, "r") as f1:
        for line in f1:
            try:
                data = json.loads(line)
                final_data = {}
                # get og from retweets
                if (data['text'].startswith('RT')):
                    num_retweets += 1
                    og_tweet = data["retweeted_status"]
                else:
                    # normal tweet
                    num_tweets += 1
                    og_tweet = data

                final_data["_id"] = og_tweet["id_str"]
                
                final_data["text"] = og_tweet["text"]
                final_data["user_id"] = og_tweet["user"]["id_str"]

                for field in tweet_fields:
                    final_data[field] = og_tweet[field]

                if og_tweet["is_quote_status"]:
                    quoted_status = {}
                    final_data["is_quote_status"] = True

                    data_level_2 = og_tweet["quoted_status"]
                    quoted_status["_id"] = data_level_2["id_str"]
                    quoted_status["text"] = data_level_2["text"]
                    quoted_status["user_id"] = data_level_2["user"]["id_str"]

                    for field in tweet_fields:
                        quoted_status[field] = data_level_2[field]

                    final_data["quoted_status"] = quoted_status
                else:
                    final_data["is_quote_status"] = False

                if final_data:
                    tweets[final_data["_id"]] = final_data

            except Exception as e:
                exceptions.append(e)

    print('num of tweets=',num_tweets, 'num of retweets=',num_retweets)
    print(len(tweets),  " tweets captured")
    print(len(exceptions), " exceptions")
    return tweets

def add_retweets(file_name, tweets):
    num_tweets=0
    num_retweets=0
    exceptions = []
    unique_ones = set()

    with open(file_name, "r") as f1:
        for line in f1:
            try:
                data = json.loads(line)
                final_data = {}
                    
                if (data['text'].startswith('RT')):
                    num_retweets += 1
                    retweets = {}

                    retweets["_id"] = data["id_str"]
                    retweets["user_id"] = data["user"]["id_str"]

                    for field in tweet_fields:
                        retweets[field] = data[field]

                    og_tweet_ref = data["retweeted_status"]["id_str"]
                        
                    final_data = tweets[og_tweet_ref]
                    if not final_data:
                        print("og ref failked", data)

                    if "retweets" not in final_data:
                        final_data["retweets"] = []

                    final_data["retweets"].append(retweets)

                    if final_data:
                        tweets[og_tweet_ref] = final_data

            except Exception as e:
                exceptions.append(e)

        print('num of tweets=',num_tweets, 'num of retweets=',num_retweets)
        print(len(tweets),  " tweets captured")
        print(len(exceptions), " exceptions")
        return tweets


In [107]:
tweets = get_og_tweets(file_name)

num of tweets= 7356 num of retweets= 11162
10984  tweets captured
18542  exceptions


In [108]:
tweets_fin = add_retweets(file_name, tweets)

num of tweets= 0 num of retweets= 11162
10984  tweets captured
18540  exceptions


In [110]:
pprint(tweets_fin['1249378751349231616'])

{'_id': '1249378751349231616',
 'created_at': 'Sun Apr 12 16:48:01 +0000 2020',
 'entities': {'hashtags': [], 'symbols': [], 'urls': [], 'user_mentions': []},
 'favorite_count': 50,
 'id_str': '1249378751349231616',
 'is_quote_status': True,
 'lang': 'en',
 'quote_count': 1,
 'quoted_status': {'_id': '1249315454797168641',
                   'created_at': 'Sun Apr 12 12:36:30 +0000 2020',
                   'entities': {'hashtags': [],
                                'symbols': [],
                                'urls': [],
                                'user_mentions': []},
                   'favorite_count': 33006,
                   'id_str': '1249315454797168641',
                   'lang': 'en',
                   'quote_count': 3376,
                   'reply_count': 1956,
                   'retweet_count': 9246,
                   'text': 'I pray my haters die of corona virus 😷',
                   'user_id': '46769281'},
 'reply_count': 1,
 'retweet_count': 24,
 'retweets'

In [111]:
def populate_tweets(tweets, client, db, collection):
    count = 0
    unique_set = set()
    for t in tweets.values():
#         if t["_id"] not in unique_set:
#             unique_set.add(t["_id"])
        result = collection.insert_one(t)
        count += 1
#             print("inserted", count)
    print(" {} Tweets inserted into MongoDB collection".format(count))

## Caution - Inserting to DB

In [112]:
populate_tweets(tweets_fin, client, db, collection)

 10984 Tweets inserted into MongoDB collection


#### Reading DB

In [113]:
database_names = client.list_database_names()

for database_name in database_names:
    print(database_name)

sample
sample_test
twitter_db
admin
local


In [114]:
collection_names = db.list_collection_names()

print("Collections in the database:")
for collection_name in collection_names:
    print(collection_name)


Collections in the database:
tweets_test


In [115]:
document_count = collection.count_documents({})
print("Number of documents in the collection:", document_count)

Number of documents in the collection: 10984


#### Search by keyword

In [116]:
keyword = "house"
query = {"text": {"$regex": keyword, "$options": "i"}}

matching_tweets = collection.find(query)

count = 0
for tweet in matching_tweets:
    count += 1
    pprint(tweet['text'])

print(" ")
print("{} tweets found".format(count))

('corona has proven to me that i can be locked in my house with nothing to do '
 'but homework and still not do homework')
('Corona got me in the house bored https://t.co/ibtjb5Z27u for more full '
 'videos https://t.co/X6QX6vYX0z')
('Karnataka medical education minister @mla_sudhakar flaunts his swanky house '
 'and pool. Picture goes viral. Most crit… https://t.co/lka9Hf4O5H')
('Chairman ppp @BBhuttoZardari meet with ex president of Pakistan ASIF ALI '
 'ZARDARI at Bilwal house &amp;  PPP President A… https://t.co/b0Li4rPtE8')
('SAVE NOW on Premium Bamboo Drawer Dividers.  Keep your house safe and '
 'healthy - touch less when utensil or clothes… https://t.co/vl4f1ZFAz8')
('@GarickPLUGs @pengmanmodel @saleem_papi @Biisi96 @Amandachindah @Mayor_ofph '
 '@Dehkunle @ogaparrot @Alex_Houseof308… https://t.co/crPkysQqFF')
('@CatherinePaiz I’m all alone this Easter... parents won’t let me in there '
 'house because of corona😭')
('@johnnjenga @StateHouseKenya too sad how the body is handled,

## SQL

In [118]:
sql_username = os.environ.get("SQLUSERNAME")
sql_password = os.environ.get("SQLPASSWORD")

conn = mysql.connector.connect(
    host = "localhost",
    user = sql_username,
    password = sql_password
)

cursor = conn.cursor()
cursor.execute("CREATE DATABASE IF NOT EXISTS twitter_user_data")
cursor.execute("USE twitter_user_data")

In [119]:
table_name = "users_info"
drop_table_query = f"DROP TABLE IF EXISTS {table_name};"

try:
    cursor.execute(drop_table_query)
    print(f"Table '{table_name}' dropped successfully.")
    conn.commit()

except mysql.connector.Error as error:
    print(f"Failed to drop table: {error}")

Table 'users_info' dropped successfully.


In [120]:
create_table_query = """
CREATE TABLE IF NOT EXISTS users_info (
    id VARCHAR(255) PRIMARY KEY,
    name VARCHAR(255),
    screen_name VARCHAR(255),
    location VARCHAR(255),
    description TEXT,
    verified BOOLEAN,
    followers_count INT,
    friends_count INT,
    created_at DATETIME
)
"""

cursor.execute(create_table_query)

In [121]:
def insert_user_data(user, cursor):

    # Insert the user data into the table
    insert_query = """
    INSERT INTO users_info (id, name, screen_name, location, description, verified, followers_count, friends_count, created_at)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    verified = 1 if user["verified"] else 0
    datetime_object = datetime.strptime(user["created_at"], '%a %b %d %H:%M:%S %z %Y').strftime('%Y-%m-%d %H:%M:%S')
    try:
        cursor.execute(insert_query, (
            user["id_str"],
            user["name"],
            user["screen_name"],
            user["location"],
            user["description"],
            verified,
            user["followers_count"],
            user["friends_count"],
            datetime_object
        ))
    except Exception as e:
        print(e)

In [117]:
def add_users(tweet, users):
    users.append(tweet["user"])
    if (tweet["text"].startswith('RT')):
        users.append(tweet["retweeted_status"]["user"])
        if tweet["retweeted_status"]["is_quote_status"]:
            users.append(tweet["retweeted_status"]["quoted_status"]["user"])
    elif tweet["is_quote_status"]:
        users.append(tweet["quoted_status"]["user"])
    return users

def get_users(file_name):
    unique_ids = set()
    user_data_unique = []
    users = []

    with open(file_name, "r") as f1:
        for line in f1:
            try:
                data = json.loads(line)
                users = add_users(data, users)
            except Exception as e:
                continue

    for user in users:
        if user["id_str"] not in unique_ids:
            unique_ids.add(user["id_str"])
            user_data_unique.append(user)

    return user_data_unique

In [11]:
user_data_unique = get_users(file_name)
print('num of unique users',len(user_data_unique), "\n\n\n")
print(user_data_unique[0])

num of unique users 19232 



{'id': 1242817830946508801, 'id_str': '1242817830946508801', 'name': 'juwelz v', 'screen_name': 'juwelz_v', 'location': 'Lower East Side, Manhattan', 'url': 'https://linktr.ee/juwelzv', 'description': 'Event Lyfe LLC .. Brand Ambassador: #visionarysociety Music Manager: @ssizzzlee & @tmoneybeatsbang Production🎥📸 #BringNyCultureBack 🌊🌊🌊 !!', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 43, 'friends_count': 118, 'listed_count': 0, 'favourites_count': 722, 'statuses_count': 906, 'created_at': 'Wed Mar 25 14:17:28 +0000 2020', 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'profile_background_color': 'F5F8FA', 'profile_background_image_url': '', 'profile_background_image_url_https': '', 'profile_background_tile': False, 'profile_link_color': '1DA1F2', 'profile_sidebar_border_color': 'C0DEED', 'profile_sidebar_fill_color': 'DDEEF6', 'profil

#### Caution Inserting to DB

In [12]:
for user in user_data_unique:
    insert_user_data(user, cursor) 

In [13]:
select_query = "SELECT count(*) FROM users_info"
query_2 = "select * from users_info where id = '1242817830946508801'"


cursor.execute(select_query)

rows = cursor.fetchall()

for row in rows:
    print(row)
    
cursor.execute(query_2)

rows = cursor.fetchall()

for row in rows:
    print(row)

(19232,)
('1242817830946508801', 'juwelz v', 'juwelz_v', 'Lower East Side, Manhattan', 'Event Lyfe LLC .. Brand Ambassador: #visionarysociety Music Manager: @ssizzzlee & @tmoneybeatsbang Production🎥📸 #BringNyCultureBack 🌊🌊🌊 !!', 0, 43, 118, datetime.datetime(2020, 3, 25, 14, 17, 28))


In [89]:
cursor.close()
conn.close()